# Tarea 8 - Luigi

_175904 - Jorge III Altamirano Astorga_

## Limpieza de Datos

In [16]:
#importo librerías
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import DataFrameStatFunctions, DataFrame
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
import pandas as pd
import re as re
import time

In [62]:
#arranque de Spark
conf = SparkConf()
conf.set("spark.driver.memory", "16g")
conf.set("spark.driver.cores", 4)
conf.set("spark.driver.memoryOverhead", 0.9)
conf.set("spark.executor.memory", "32g")
conf.set("spark.executor.cores", 12)
conf.set("spark.jars", "/home/jaa6766")
sc = SparkContext(master = "local[14]", sparkHome="/usr/local/spark/", 
                  appName="tarea-mge-8", conf=conf)
spark = SQLContext(sc)

### Carga de Datos

In [64]:
%%time
data = spark.read.csv("hdfs://172.17.0.2:9000/data/profeco/data.csv", 
                      schema = StructType() \
                        .add("producto", StringType(), False) \
                        .add("presentacion", StringType(), True) \
                        .add("marca", StringType(), True) \
                        .add("categoria", StringType(), True) \
                        .add("catalogo", StringType(), True) \
                        .add("precio", DecimalType(precision=16, scale=4), True) \
                        .add("fechaRegistro", TimestampType(), True) \
                        .add("cadenaComercial", StringType(), True) \
                        .add("giro", StringType(), True) \
                        .add("nombreComercial", StringType(), True) \
                        .add("direccion", StringType(), True) \
                        .add("estado", StringType(), True) \
                        .add("municipio", StringType(), True) \
                        .add("latitud", DoubleType(), True) \
                        .add("longitud", DoubleType(), True),
                      inferSchema=False, 
                      escape='"',
                      quote='"',
                      timestampFormat="yyyy-MM-dd hh:mm:ss",
                      header=True)
data.write.parquet("hdfs://172.17.0.2:9000/data/profeco/data.parquet", mode="overwrite")
data = spark.read.parquet("hdfs://172.17.0.2:9000/data/profeco/data.parquet")
data.cache()
data.show(2)
data.printSchema()

+--------------------+--------------------+-----+--------------------+--------+------+-------------------+---------------+--------------------+--------------------+--------------------+----------------+-------------------+---------+----------+
|            producto|        presentacion|marca|           categoria|catalogo|precio|      fechaRegistro|cadenaComercial|                giro|     nombreComercial|           direccion|          estado|          municipio|  latitud|  longitud|
+--------------------+--------------------+-----+--------------------+--------+------+-------------------+---------------+--------------------+--------------------+--------------------+----------------+-------------------+---------+----------+
|                null|                null| null|                null|    null|  null|               null|           null|                null|                null|                null|            null|               null|     null|      null|
|POLVO P/PREPARAR ...|SO

### Sumario

In [ ]:
%%time
#sample = data.sample(fraction = 0.0001, seed=175904)
summary = data.summary()

CPU times: user 17.5 s, sys: 4.71 s, total: 22.2 s
Wall time: 23h 9min 20s


In [13]:
summary.coalesce(1).write.parquet("hdfs://172.17.0.2:9000/data/profeco/summary.parquet", mode="overwrite")

In [11]:
for (nombre, tipo) in data.dtypes:
    print("####### %s #######"%nombre)
    if tipo in "string":
        summary.select("summary", nombre) \
            .filter("summary = 'min' OR summary = 'max'") \
            .show(truncate=False)
    elif "decimal" in tipo or tipo in "double":
        summary.select("summary", nombre) \
            .filter("summary <> 'count'") \
            .show(truncate=False)
    else:
        print("Sin sumario\n\n")

####### producto #######
+-------+--------+
|summary|producto|
+-------+--------+
|min    |A.S.COR |
|max    |ZYPREXA |
+-------+--------+

####### presentacion #######
+-------+----------------------------------------------+
|summary|presentacion                                  |
+-------+----------------------------------------------+
|min    |# 6017872 O C.B. 96911. HEDBANZ ¡PARA ADULTOS!|
|max    |´0280808. PATIN/SCOOTER. ULTIMATE SPIDER-MAN  |
+-------+----------------------------------------------+

####### marca #######
+-------+------------------------+
|summary|marca                   |
+-------+------------------------+
|min    |'PAPER MATE. ERASER-MATE|
|max    |iO MABE                 |
+-------+------------------------+

####### categoria #######
+-------+---------------------+
|summary|categoria            |
+-------+---------------------+
|min    |ACCESORIOS DOMESTICOS|
|max    |VINOS Y LICORES      |
+-------+---------------------+

####### catalogo #######
+-------+--

In [132]:
%%time 
#funcion que hace lo mismo que summary, pero 
#irónicamente más rápido que spark (esta es para numéricos)
def summary_j3a(col):
    min1 = data.select(min(data[col]).alias("min")).toPandas().transpose()
    max1 = data.select(max(data[col]).alias("max")).toPandas().transpose()
    avg1 = data.select(mean(data[col]).alias("avg")).toPandas().transpose()
    std1 = data.select(stddev(data[col]).alias("stddev")).toPandas().transpose()
    probs = [0.25, 0.5, 0.75]
    qnt1 = pd.DataFrame(  \
        data.approxQuantile(col, probabilities=probs, relativeError=0.05)
    )
    qnt1 = qnt1.rename_axis({0: "25%", 1: "50%", 2: "75%"}, axis=0)
    complete = min1.append(qnt1).append(max1).append(avg1).append(std1)
    complete = complete.rename(index=str, columns={0: col})
    complete[col] = complete.apply(lambda x: "{:,}".format(x[col]), axis=1)
    return complete
#funcion que hace lo mismo que summary, pero 
#irónicamente más rápido que spark (esta es para strings)
def summary_string_j3a(col):
    min1 = data.select(min(data[col]).alias("min")).toPandas().transpose()
    max1 = data.select(max(data[col]).alias("max")).toPandas().transpose()
    complete = min1.append(max1)
    complete = complete.rename(index=str, columns={0: col})
    return complete
summary_list = list()
#ahora lo vamos a hacer por cada columna
for (nombre, tipo) in data.dtypes:
    ## header de la columna
    print("####### %s #######"%nombre)
    ## los conteos los hacemos una vez
    cnt1 = pd.DataFrame([{"count": data.count()}]).transpose()
    cnt1 = cnt1.rename(index=str, columns={0: nombre})
    ## sacar output de una columna string
    if tipo in "string":
        ret_val = cnt1.append(summary_string_j3a(nombre))
        #agregarlo a un pandas para escritura al final
        summary_list.append(ret_val)
        print(ret_val)
    ## sacar output de una columna numérica
    elif "decimal" in tipo or tipo in "double":
        #agregarlo a un pandas para escritura al final
        ret_val = cnt1.append(summary_j3a(nombre))
        summary_list.append(ret_val)
        print(ret_val)
    #no sabemos qué es, entoncess sacamos sin datos :-)
    #se puede mejorar
    else:
        print("Sin sumario")
    print("\n")
sumario = pd.concat(summary_list, 1)
sumario_df = spark.createDataFrame(sumario.astype(str))
sumario_df.coalesce(1).write.parquet("hdfs://172.17.0.2:9000/data/profeco/sumario_df.parquet", 
                                    mode="overwrite")
sumario_df.show()

####### producto #######
       producto
count  62530715
min     A.S.COR
max     ZYPREXA


####### presentacion #######
                                         presentacion
count                                        62530715
min    # 6017872 O C.B. 96911. HEDBANZ ¡PARA ADULTOS!
max      ´0280808. PATIN/SCOOTER. ULTIMATE SPIDER-MAN


####### marca #######
                          marca
count                  62530715
min    'PAPER MATE. ERASER-MATE
max                     iO MABE


####### categoria #######
                   categoria
count               62530715
min    ACCESORIOS DOMESTICOS
max          VINOS Y LICORES


####### catalogo #######
               catalogo
count          62530715
min             BASICOS
max    UTILES ESCOLARES


####### precio #######
                     precio
count              62530715
min                  0.1000
25%                    21.9
50%                    45.9
75%                   289.0
max            299,999.0000
avg            520.13878

### Cuantiles y Conteos

In [55]:
%%time
data.approxQuantile("precio", probabilities=[0.1, 0.25, 0.5, 0.85, 0.9], relativeError=0.01)

CPU times: user 28.4 ms, sys: 9.64 ms, total: 38.1 ms
Wall time: 4.47 s


[10.89, 18.5, 39.9, 459.0, 924.0]

In [56]:
%%time
data.select(approxCountDistinct(data.producto, rsd=0.01)).show()
print(data.count())

+-------------------------------+
|approx_count_distinct(producto)|
+-------------------------------+
|                           1099|
+-------------------------------+

62530715
CPU times: user 6.44 ms, sys: 4.48 ms, total: 10.9 ms
Wall time: 3.2 s


In [57]:
data.select(approxCountDistinct(data.estado, rsd=0.01)).show()

+-----------------------------+
|approx_count_distinct(estado)|
+-----------------------------+
|                           32|
+-----------------------------+



In [58]:
%%time
data.select(data.estado).distinct().orderBy(data.estado).show(35)

+--------------------+
|              estado|
+--------------------+
|                null|
|      AGUASCALIENTES|
|     BAJA CALIFORNIA|
| BAJA CALIFORNIA SUR|
|            CAMPECHE|
|             CHIAPAS|
|           CHIHUAHUA|
|COAHUILA DE ZARAGOZA|
|              COLIMA|
|    DISTRITO FEDERAL|
|             DURANGO|
|          GUANAJUATO|
|            GUERRERO|
|             HIDALGO|
|             JALISCO|
| MICHOACÁN DE OCAMPO|
|             MORELOS|
|              MÉXICO|
|             NAYARIT|
|          NUEVO LEÓN|
|              OAXACA|
|              PUEBLA|
|           QUERÉTARO|
|        QUINTANA ROO|
|     SAN LUIS POTOSÍ|
|             SINALOA|
|              SONORA|
|             TABASCO|
|          TAMAULIPAS|
|            TLAXCALA|
|VERACRUZ DE IGNAC...|
|             YUCATÁN|
|           ZACATECAS|
+--------------------+

CPU times: user 8.96 ms, sys: 2.97 ms, total: 11.9 ms
Wall time: 1.68 s


### Datos Nulos

Desconocemos porqué quedaron tantos datos nulos. Esto nos resalto al encontrar $estado == Null$.

In [133]:
print("Número de estados con valor nulo: %d\n\n"%data.filter("estado is null").count())
data.filter("estado is null").show(5)

Número de estados con valor nulo: 1130352


+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|producto|presentacion|marca|categoria|catalogo|precio|fechaRegistro|cadenaComercial|giro|nombreComercial|direccion|estado|municipio|latitud|longitud|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|    null|        null| null|     null|    null|  null|         null|           null|null|           null|     null|  null|     null|   null|    null|
|    null|        null| null|     null|    null|  null|         null|           null|null|           null|     null|  null|     null|   null|    null|
|    null|        null| null|     null|    null|  null|         null|           null|null|           null|     null|  null|     null|   null|    null|
|    null|        null| null|     null|    null|  

In [60]:
sc.stop()